In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -o /content/drive/MyDrive/dataset/archive.zip -d HairType

Archive:  /content/drive/MyDrive/dataset/archive.zip
  inflating: HairType/data/Straight/000028d73-Nicole_Scherzinger.jpg  
  inflating: HairType/data/Straight/01-10-2019-latest-haircut-for-girls_Equal_Length_4PNG.jpg  
  inflating: HairType/data/Straight/01-10-2019-latest-haircut-for-girls_short_pixie_straight_undercut_3PNG.jpg  
  inflating: HairType/data/Straight/01b8ef4743e2a5d1b3e888ef18acdfd0.jpg  
  inflating: HairType/data/Straight/03e7316208c63582103294f3e1ef8b7b.jpg  
  inflating: HairType/data/Straight/051c0f399232bd22fb8ff55a6dde5441.jpg  
  inflating: HairType/data/Straight/060c671d89cba47235d3a8a620bed080.jpg  
  inflating: HairType/data/Straight/06faceb8ea130fb976a3339f52ba611d--hair-products-for-men-gentlemens-guide.jpg  
  inflating: HairType/data/Straight/0a1c060ba84f25dd6f353656818fb646.jpg  
  inflating: HairType/data/Straight/0a7bec97b58f02f16f1da241ed4d99dd.jpg  
  inflating: HairType/data/Straight/0f9d3172f6ec3206d6f76318830159b2.jpg  
  inflating: HairType/data/

In [ ]:
dataset = '/content/HairType/data'

In [ ]:
import warnings
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import models,layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from PIL import Image
import hashlib
import os

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
# Fungsi untuk memeriksa apakah gambar bisa dibuka
def is_image_valid(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()  # Verifikasi apakah gambar valid
        return True
    except (IOError, SyntaxError):
        return False

# Fungsi untuk menghasilkan hash dari gambar
def generate_image_hash(image_path):
    with open(image_path, 'rb') as f:
        img_hash = hashlib.md5(f.read()).hexdigest()
    return img_hash

In [ ]:
# Fungsi untuk menghapus gambar rusak dan duplikat
def clean_image_data(directory):
    valid_images = []
    image_hashes = set()
    for subdir, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(subdir, file)

            # Mengecek apakah file adalah gambar
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp')):
                # Mengecek gambar rusak
                if not is_image_valid(file_path):
                    print(f"Gambar rusak ditemukan dan dihapus: {file_path}")
                    os.remove(file_path)
                else:
                    # Mengecek gambar duplikat
                    img_hash = generate_image_hash(file_path)
                    if img_hash in image_hashes:
                        print(f"Gambar duplikat ditemukan dan dihapus: {file_path}")
                        os.remove(file_path)
                    else:
                        image_hashes.add(img_hash)
                        valid_images.append(file_path)

    print(f"Total gambar yang valid: {len(valid_images)}")
    return valid_images

# Praproses data gambar
directory = dataset  # Ganti dengan path dataset Anda
valid_images = clean_image_data(directory)


Gambar duplikat ditemukan dan dihapus: /content/HairType/data/curly/image42.jpg
Gambar duplikat ditemukan dan dihapus: /content/HairType/data/curly/image277.jpg
Gambar duplikat ditemukan dan dihapus: /content/HairType/data/Wavy/image3.jpg
Gambar duplikat ditemukan dan dihapus: /content/HairType/data/Wavy/image67.jpg
Gambar duplikat ditemukan dan dihapus: /content/HairType/data/Wavy/image6(2).jpeg
Gambar duplikat ditemukan dan dihapus: /content/HairType/data/Straight/images (10).jpg
Gambar duplikat ditemukan dan dihapus: /content/HairType/data/Straight/images124.jpg
Gambar duplikat ditemukan dan dihapus: /content/HairType/data/Straight/images (3).jpg
Gambar duplikat ditemukan dan dihapus: /content/HairType/data/Straight/image6.jpeg
Total gambar yang valid: 1978


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)

In [ ]:
training_set = train_datagen.flow_from_directory(dataset, target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset = 'training')
testing_set = train_datagen.flow_from_directory(dataset, target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset = 'validation')

Found 1584 images belonging to 5 classes.
Found 394 images belonging to 5 classes.


In [ ]:
base_model = MobileNetV2(input_shape = (224,224,3), weights = 'imagenet', include_top = False)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
base_model.trainable = False
cnn = models.Sequential([
    base_model,
# Pooling Layer.
    layers.GlobalAveragePooling2D(),
# Fully Connected Layer.
    layers.Dense(units = 128, activation = 'relu'),
# Output Layer.
    layers.Dense(units = 5, activation = 'softmax')
])

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
prediction = cnn.fit(training_set, validation_data = testing_set, epochs = 10)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step - accuracy: 0.5622 - loss: 1.1325 - val_accuracy: 0.7792 - val_loss: 0.7078
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.8480 - loss: 0.4084 - val_accuracy: 0.8096 - val_loss: 0.5795
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.9073 - loss: 0.2574 - val_accuracy: 0.7741 - val_loss: 0.6026
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.9382 - loss: 0.1906 - val_accuracy: 0.8198 - val_loss: 0.5354
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.9767 - loss: 0.1243 - val_accuracy: 0.8350 - val_loss: 0.5674
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.9797 - loss: 0.0932 - val_accuracy: 0.8350 - val_loss: 0.5645
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - accuracy: 0.9961 - loss: 0.0628 - val_accuracy: 0.8223 - val_loss: 0.6007
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.9984 - loss: 0.0476 - val_accuracy: 0.8350 - val

In [ ]:
loss_of_model, accuracy_of_model = cnn.evaluate(testing_set)
print("Validation Accuracy: ",accuracy_of_model*100)

13/13 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.8486 - loss: 0.5465
Validation Accuracy:  82.74111747741699


In [ ]:
cnn.export('saved_model/kabe')  # Menghasilkan direktori SavedModel

Saved artifact at 'saved_model/kabe'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  137402578693888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137402578776160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137402578778096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137402578776512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137402578777568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137402578775456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137402530075344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137402530076400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137402530072528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137402530075168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1374025786

In [ ]:
cnn.save('kabe.h5')

In [ ]:
# Konversi dari SavedModel
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model/kabe')
tflite_model = converter.convert()

# Simpan model TFLite
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)